# 1. Gestión del dataset - Estratificación automática

Cargar CSV original y visualizar los datos. Verificamos que las columnas clave como bcn_filename, diagnosis y lesion_id están presentes en el archivo CSV

In [31]:
import pandas as pd

# Ruta a tu CSV
csv_path = r"C:\BCN20000\bcn_20k_train.csv"

# Cargamos el CSV
df = pd.read_csv(csv_path)

# Mostramos las primeras filas para revisar la estructura
df.head()

,bcn_filename,age_approx,anatom_site_general,diagnosis,lesion_id,capture_date,sex,split
0,BCN_0000000001.jpg,55.0,anterior torso,MEL,BCN_0003884,2012-05-16,male,train
1,BCN_0000000003.jpg,50.0,anterior torso,MEL,BCN_0000019,2015-07-09,female,train
2,BCN_0000000004.jpg,85.0,head/neck,SCC,BCN_0003499,2015-11-23,male,train
3,BCN_0000000006.jpg,60.0,anterior torso,NV,BCN_0003316,2015-06-16,male,train
4,BCN_0000000010.jpg,30.0,anterior torso,BCC,BCN_0004874,2014-02-18,female,train


Añadir la ruta completa a cada imagen

In [32]:
# Ruta a la carpeta donde están todas las imágenes mezcladas
img_dir = r"C:\BCN20000\BCN_20k_train\bcn_20k_train"

# Añadimos la columna con la ruta completa
df["filepath"] = df["bcn_filename"].apply(lambda x: f"{img_dir}\{x}")
df = df[["filepath", "diagnosis", "lesion_id"]]

# Verificamos
df.head()

,filepath,diagnosis,lesion_id
0,C:\BCN20000\BCN_20k_train\bcn_20k_train\BCN_00...,MEL,BCN_0003884
1,C:\BCN20000\BCN_20k_train\bcn_20k_train\BCN_00...,MEL,BCN_0000019
2,C:\BCN20000\BCN_20k_train\bcn_20k_train\BCN_00...,SCC,BCN_0003499
3,C:\BCN20000\BCN_20k_train\bcn_20k_train\BCN_00...,NV,BCN_0003316
4,C:\BCN20000\BCN_20k_train\bcn_20k_train\BCN_00...,BCC,BCN_0004874


Asignamos etiquetas numéricas a cada diagnóstico

In [33]:
# Asignamos un número único a cada diagnóstico
label_map = {label: idx for idx, label in enumerate(sorted(df["diagnosis"].unique()))}
df["label"] = df["diagnosis"].map(label_map)

print("Etiquetas asignadas:")
print(label_map)

Etiquetas asignadas:
{'AK': 0, 'BCC': 1, 'BKL': 2, 'DF': 3, 'MEL': 4, 'NV': 5, 'SCC': 6, 'VASC': 7}


Dividir de forma estratificada sin mezclar lesion_id

Usamos StratifiedGroupKFold para que no haya imágenes de la misma lesion_id en splits diferentes.

In [34]:
from sklearn.model_selection import StratifiedGroupKFold

# Creamos el objeto de KFold estratificado por grupo (lesion_id)
sgkf = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=42)

# Obtenemos los índices
for train_val_idx, test_idx in sgkf.split(df, df["label"], groups=df["lesion_id"]):
    break  # Solo queremos una partición (1 split)

train_val_df = df.iloc[train_val_idx]
test_df = df.iloc[test_idx]

Ahora hacemos otro split dentro del conjunto train_val_df para obtener validación

In [35]:
from sklearn.model_selection import train_test_split

# Dividimos entrenamiento y validación
train_df, val_df = train_test_split(
    train_val_df,
    test_size=0.1,  # 10% de validación
    stratify=train_val_df["label"],
    random_state=42
)

# Mostramos tamaños
print(f"Train: {len(train_df)}")
print(f"Validation: {len(val_df)}")
print(f"Test: {len(test_df)}")

Train: 8971
Validation: 997
Test: 2445


Guardar CSVs para usar en TensorFlow

In [36]:
# Guardamos los splits como CSVs
train_df.to_csv("train_split.csv", index=False)
val_df.to_csv("val_split.csv", index=False)
test_df.to_csv("test_split.csv", index=False)

print("Splits guardados con éxito.")

Splits guardados con éxito.


Acabamos de crear tres archivos CSV (train_split.csv, val_split.csv, test_split.csv) que apuntan a las rutas absolutas de las imágenes, sin mover, copiar ni alterar nada en la carpeta original BCN20000.

Evitamos tocar el dataset original: la carpeta con imágenes mezcladas queda intacta.

Evitamos duplicados o inconsistencias: cada split es coherente y sin data leakage (gracias a la separación por lesion_id).

Compatibilidad con TensorFlow.